# Build a Basic Bot

In [1]:
import time
import random

In [2]:
def answer(user_input):
    """Take the user's input and respond in kind."""
    resp = f"{user_input} back at ya!"
    time.sleep(0.5)
    return resp
answer("Heya")

'Heya back at ya!'

In [3]:
def user_speaks(user_input, user_format="USER:", bot_format="BOT:"):
    """Passes the user's input to response handler."""
    print(f"{user_format} {user_input}")
    resp = answer(user_input)
    return f"{bot_format} {resp}"

user_speaks("Yoyoyo!")


USER: Yoyoyo!


'BOT: Yoyoyo! back at ya!'

## Basic Chat

Introduces some stock responses for exact question matches. Adds in some limited variation.

### Stock Answers

In [4]:
food = "ice cream"
weather = "overcast"
series = "The Rings of Power"
stock_qs = {
    "what's your favourite food?": f"I really like {food}.",
    "how's the weather today?": f"It's looking {weather} right now.",
    "what are you currently watching?": f"{series}, it's really great!"

}

In [5]:
def stock_answer(user_input, user_format="USER:", bot_format="BOT:"):
    """Take the user's input, check for stock responses, respond in kind
    if no matches found."""
    if user_input.lower() in stock_qs.keys():
        time.sleep(0.5)
        resp = stock_qs[user_input]
    else:
        resp = answer(user_input)
    return resp


In [6]:
def user_speaks(user_input, user_format="USER:", bot_format="BOT:"):
    """Passes the user's input to response handler."""
    print(f"{user_format} {user_input}")
    resp = stock_answer(user_input)
    return f"{bot_format} {resp}"

In [7]:
user_speaks("what are you currently watching?")

USER: what are you currently watching?


"BOT: The Rings of Power, it's really great!"

In [8]:
user_speaks("That's rubbish!")

USER: That's rubbish!


"BOT: That's rubbish! back at ya!"

### Variation

In [9]:
stock_qs = {
    "what's your favourite food?": [
        f"I really like {food}.",
        f"{food} is my favourite.",
        f"I'm addicted to {food}."
        ],
    "how's the weather today?": [
        f"It's looking {weather} right now.",
        f"Today it's mainly {weather}."
        ],
    "what are you currently watching?": [
        f"{series}, it's really great!",
        f"I just can't get enough of {series} right now.",
        f"Way too much {series}."
        ]

}

In [10]:
def stock_answer(user_input, user_format="USER:", bot_format="BOT:"):
    """Take the user's input, check for stock responses, respond in kind
    if no matches found."""
    if user_input.lower() in stock_qs.keys():
        time.sleep(0.5)
        resp = random.choice(stock_qs[user_input])
    else:
        resp = answer(user_input)
    return resp

In [11]:
print(user_speaks("what are you currently watching?"))
print(user_speaks("what are you currently watching?"))
print(user_speaks("what are you currently watching?"))


USER: what are you currently watching?
BOT: Way too much The Rings of Power.
USER: what are you currently watching?
BOT: Way too much The Rings of Power.
USER: what are you currently watching?
BOT: Way too much The Rings of Power.


## Asking Qs

In [12]:
is_question = {
    True: [
        "Dunno!",
        "Ummm...",
        "Don't have a scooby!"
    ],
    False: [
        "How are you today :) ?",
        "Did you remember to feed the dogs?",
        "Where did you go on hols?"
    ]
}

In [13]:
def stock_answer(user_input, user_format="USER:", bot_format="BOT:"):
    """Take the user's input, check for stock responses, respond in kind
    if no matches found."""
    if user_input.lower() in stock_qs.keys():
        time.sleep(0.5)
        resp = random.choice(stock_qs[user_input])
    else:
        # Replace the stock answer with the q check
        resp = random.choice(is_question[user_input.endswith("?")])
    return resp

In [14]:
print(user_speaks("what are you currently watching?"))
print(user_speaks("How are you feeling?"))
print(user_speaks("How are you feeling?"))
print(user_speaks("I'm feeling great thanks."))
print(user_speaks("I love cake."))

USER: what are you currently watching?
BOT: Way too much The Rings of Power.
USER: How are you feeling?
BOT: Ummm...
USER: How are you feeling?
BOT: Ummm...
USER: I'm feeling great thanks.
BOT: Where did you go on hols?
USER: I love cake.
BOT: Did you remember to feed the dogs?


## Regular Expressions

An interesting use of groups to pull out matching clauses when a match has been found, without needing to do the positive lookahead dance... By a group, I mean:

pattern = "foo (.*)"
string = "foo bar"
By calling the group method on the match object, you can either access the entire matched string (`.group(0)`) or just the clause matched within the group parentheses (`.group(1)` == "bar").

### Matching questions with responses


In [15]:
# define a dictionary with some regex keys to use as matches. In this example, f strings are dropped
# in favour of format statements with the {0} placeholder

q_regexes = {
    'Can you (.*)': [
        'Sometimes I can {0}',
        'I {0} only if the feeling takes me',
        "Nope, never!"
        ],
 "Why can't I (.*)": [
    'Is {0} what you really need?',
    "How will {0} change your life?",
    "Don't worry about {0}",
    "Don't worry about it!"
    ]
  }

In [16]:
import re

In [17]:
def choose_answer(match_rules, input_q):
    """Searches the keys in rules for a match, if found,
    format a random response with part of the users input_q.
    Sneaky eh?"""
    selected_resp = "default"

    for pat, resps in match_rules.items():
        search_res = re.search(pat, input_q)
        if search_res is not None:
            selected_resp = random.choice(resps)
            # This bit handles the formatting of the placeholder {0}
            if "{0}" in selected_resp:
                selected_resp = selected_resp.format(search_res.group(1))
    return selected_resp


In [18]:
print(choose_answer(q_regexes, "Can you swim"))
print(choose_answer(q_regexes, "Can you swim"))
print(choose_answer(q_regexes, "Can you swim"))
print(choose_answer(q_regexes, "Why can't I sing?"))
print(choose_answer(q_regexes, "Why can't I sing?"))
print(choose_answer(q_regexes, "Why can't I sing?"))

I swim only if the feeling takes me
Sometimes I can swim
Nope, never!
Don't worry about it!
Don't worry about sing?
Don't worry about it!


## Grammar

Tense of sentences is important. Talks about conjugating verbs but doesn't actually do this. Instead, pronouns are updated to switch subject of a clause, eg -

"My best suit" becomes "Your best suit".

In [39]:
def switch_pronouns(pronoun_message):
    """Toggles any regex matched pronouns"""
    # lower the message to avoid any case sensitivity
    m = pronoun_message.lower()
    # use if statements to check for conditions, otherwise undoes the subs
    if "i" in m:
        m = re.sub("i", "you", m)
        return m
    if "my" in m:
        m = re.sub("my", "your", m)
        return m
    if "you" in m:
        m = re.sub("your", "my", m)
        return m


In [40]:
print(switch_pronouns("I love my picture of me"))
# now only the first if statement gets executed
# need to use a stricter pattern for matching I... lol
print(switch_pronouns("Your cape"))

you love my pyoucture of me
my cape
